---
description: action list
output-file: fnctions.html
title: functions

---

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp functions

In [ ]:
#| export

import os
import shutil

def dispatch(app_name:str):
    try:
        from win32com import client
        app = client.gencache.EnsureDispatch(app_name)
    except AttributeError:
        # Corner case dependencies.
        import os
        import re
        import sys
        import shutil
        # Remove cache and try again.
        MODULE_LIST = [m.__name__ for m in sys.modules.values()]
        for module in MODULE_LIST:
            if re.match(r'win32com\.gen_py\..+', module):
                del sys.modules[module]
        shutil.rmtree(os.path.join(os.environ.get('LOCALAPPDATA'), 'Temp', 'gen_py'))
        from win32com import client
        app = client.gencache.EnsureDispatch(app_name)
    return app

In [ ]:
#| export
from pathlib import Path

def get_absolute_path(path):
    name = Path(path)
    return name.absolute().as_posix()

In [ ]:
#| export 
import importlib.resources

def get_dll_path(package_name, dll_filename):
    try:
        with importlib.resources.path(package_name, dll_filename) as dll_path:
            return str(dll_path)
    except FileNotFoundError as e:
        raise FileNotFoundError(f"The DLL file '{dll_filename}' was not found in the package '{package_name}'.") from e


In [ ]:
#| export

import winreg
import os


def add_dll_to_registry(dll_path, key_path):
    try:
        # Connect to the registry and open the specified key
        registry_key = winreg.OpenKey(winreg.HKEY_CURRENT_USER, key_path, 0, winreg.KEY_SET_VALUE)

        # Set the value for the new registry entry as a string (REG_SZ)
        winreg.SetValueEx(registry_key, "FilePathCheckerModule", 0, winreg.REG_SZ, dll_path)

        # Close the registry key
        winreg.CloseKey(registry_key)
        print("DLL path added to registry as a string value successfully.")
    except WindowsError as e:
        print("Error while adding DLL path to registry: ", e)
        
        
def get_registry_value(key_path, value_name):
    try:
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, key_path) as key:
            value, value_type = winreg.QueryValueEx(key, value_name)
            return value
    except FileNotFoundError:
        return None


In [ ]:
#| export

def check_dll():
    dll_path = get_dll_path("HwpApi", "FilePathCheckerModuleExample.dll")
    key_path = "SOFTWARE\\HNC\\HwpAutomation\\Modules"
    value_name = "FilePathCheckerModule" 

    value = get_registry_value(key_path, value_name)

    if value is None:
        add_dll_to_registry(dll_path, key_path)
    return True


In [ ]:
#| export 

def get_value(dict_, key):
    try:
        return dict_[key]
    except KeyError:
        raise KeyError(f"{key}를 해당하는 키 중 찾을 수 없습니다. 키는 {', '.join(dict_.keys())} 중에 있어야 합니다.")

In [ ]:
#| export

def set_charshape(
    charshape, 
    face_name:str=None, 
    font_type:int=None, 
    size:int=None,
    ratio:int=None, 
    spacing:int=None, 
    offset:int=None,
    bold:bool=None, 
    italic:bool=None, 
    small_caps:bool=None,
    emboss:bool=None,
    super_script:bool=None,
    sub_script:bool=None,
    underline_type:int=None,
    outline_type:int=None,
    text_color=None,
    shade_color=None,
    underline_shape:int=None,
    underline_color=None,
    shadow_offset_x:int=None,
    shadow_offset_y:int=None,
    shadow_color=None,
    strike_out_type=None,
    diac_sym_mark=None,
    use_font_space=None,
    use_kerning=None,
    height:int=None,
):
    """
    CharShape값을 입력하기 위한 함수입니다.
    `BorderFill`은 별도로 "BorderFill"타입을 사용하기 때문에 제외하였습니다. 
    """
    params = []
    categories = ["Hangul", "Latin", "Hanja", "Japanese", "Other", "Symbol", "User"]
    
    params += [("FaceName"+cat, face_name) for cat in categories] if face_name else []
    params += [("FontType"+cat, font_type) for cat in categories] if face_name else []
    params += [("Size"+cat, size) for cat in categories] if size else []
    params += [("Ratio"+cat, ratio) for cat in categories] if ratio else []
    params += [("Spacing"+cat, spacing) for cat in categories] if spacing else []
    params += [("Offset"+cat, offset) for cat in categories] if offset else []
    
    params += list(
        filter(lambda x: x[1] is not None, 
            [
                ("Bold", bold),
                ("Italic", italic),
                ("SmallCaps", small_caps),
                ("Emboss", emboss),
                ("SuperScript", super_script),
                ("SubScript", sub_script),
                ("UnderlineType", underline_type),
                ("OutlineType", outline_type),
                ("TextColor", text_color),
                ("ShadeColor", shade_color),
                ("UnderlineShape", underline_shape),
                ("UnderlineColor", underline_color),
                ("ShadowOffsetX", shadow_offset_x),
                ("ShadowOffsetY", shadow_offset_y),
                ("ShadowColor", shadow_color),
                ("StrikeOutType", strike_out_type),
                ("DiacSymMark", diac_sym_mark),
                ("UseFontSpace", use_font_space),
                ("UseKerning", use_kerning),
                ("Height", height),
            ]
        )
    ) 
    for key, value in params:
            setattr(charshape, key, value)
    
    return charshape


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()